In [328]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [329]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 시드값

In [330]:
SEED = 42

- 데이터 불러오기

In [331]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
submit_proba = pd.read_csv(f"{DATA_PATH}store_submission.csv")

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [332]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_2_삭제완료.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_2_삭제완료.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 492), (12225, 492))

# 결측치 처리

In [333]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
일별구매횟수표준편차,664
3회이상count,4214
총_생식품_구매가격,9711
평균_생식품_구매가격,1574
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [334]:
train_ft['일별구매횟수표준편차'].isnull().sum()

664

In [335]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
일별구매횟수표준편차,443
3회이상count,3284
총_생식품_구매가격,7820
평균_생식품_구매가격,1257
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [336]:

train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)
train_ft["총_생식품_구매가격"] = train_ft["총_생식품_구매가격"].fillna(0)
train_ft["평균_생식품_구매가격"] = train_ft["평균_생식품_구매가격"].fillna(0)
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
train_ft['일별구매횟수표준편차'] = train_ft['일별구매횟수표준편차'].fillna(0)


test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)
test_ft["총_생식품_구매가격"] = test_ft["총_생식품_구매가격"].fillna(0)
test_ft["평균_생식품_구매가격"] = test_ft["평균_생식품_구매가격"].fillna(0)
test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)
test_ft['일별구매횟수표준편차'] = test_ft['일별구매횟수표준편차'].fillna(0)


In [337]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [338]:
train_ft['가족단위구매비율'] = (train_ft['대분류_아동_cnt'] + train_ft['유아관련count']) / train_ft['구매건수']
test_ft['가족단위구매비율'] = (test_ft['대분류_아동_cnt'] + test_ft['유아관련count']) / test_ft['구매건수']

In [339]:
# new_columns = pd.DataFrame({
#     # '일용잡화_최소구매액_bin': ((train_ft["중_pivot_횟수비율_일용잡화"] <= 2) & (train_ft["최소구매액_bin"] >= 3.5)).astype(int),
#     # 다른 추가 열이 필요하면 이곳에 함께 정의
# })

# train_ft = pd.concat([train_ft, new_columns], axis=1)

# new_columns = pd.DataFrame({
#     # '일용잡화_최소구매액_bin': ((test_ft["중_pivot_횟수비율_일용잡화"] <= 2) & (test_ft["최소구매액_bin"] >= 3.5)).astype(int),
# })
# test_ft = pd.concat([test_ft,new_columns], axis = 1)

In [340]:
train_ft.drop('유아관련count', axis=1, inplace=True)
train_ft.drop("유아관련유무", axis=1, inplace=True)
test_ft.drop('유아관련count', axis=1, inplace=True)
test_ft.drop("유아관련유무", axis=1, inplace=True)

#11-10 추가피처

In [341]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
train_tr['구매일'] = train_tr['구매일시'].dt.date
test_tr['구매일'] = test_tr['구매일시'].dt.date

grouped = train_tr.groupby(['ID', '구매일'])['구매일시']

# 2. 각 그룹에서 최소값과 최대값을 구하고, 그 차이를 계산하여 일별 체류시간을 구합니다.
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()

# 3. ID별로 일별 체류시간의 평균을 계산합니다.
average_stay_time = daily_stay_time.groupby('ID')['체류시간'].mean()


train_ft = train_ft.merge(average_stay_time, how='left',on="ID")

grouped = test_tr.groupby(['ID', '구매일'])['구매일시']
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()
average_stay_time = daily_stay_time.groupby('ID')['체류시간'].mean()
test_ft = test_ft.merge(average_stay_time, how='left',on="ID")
train_ft["구매당체류시간"] = (train_ft["체류시간"]/3600)/train_ft["거래횟수"]
test_ft["구매당체류시간"] = (test_ft["체류시간"]/3600)/test_ft["거래횟수"]

In [342]:
agg_list=[
        ('월화수방문비율', lambda x : np.mean(x.dt.weekday < 3)),
        ('목금토일방문비율', lambda x : np.mean(x.dt.weekday >= 3))
]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

In [343]:
# arr = ['주중방문비율','주말방문비율','금오후토일방문비율','금토일방문비율']
# for col in arr:
#     train_ft.drop(col,axis=1,inplace=True)
#     test_ft.drop(col,axis=1,inplace=True)

In [344]:
train_ft

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,최소구매액_중분류수_3_4,최소구매액_중분류수_4_1,최소구매액_중분류수_4_2,최소구매액_중분류수_4_3,최소구매액_중분류수_4_4,가족단위구매비율,체류시간,구매당체류시간,월화수방문비율,목금토일방문비율
0,train_0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0,0,0,0,0,0.176471,474.545455,0.008239,0.400000,0.600000
1,train_1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,0,0,0,0,0,0.047619,2028.571429,0.013744,0.523810,0.476190
2,train_2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,0,0,0,0,0,0.205607,1549.285714,0.004219,0.587719,0.412281
3,train_3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,0,0,0,0,0,0.355330,3161.086957,0.004597,0.369668,0.630332
4,train_4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,0,0,0,0,0,0.265306,1800.000000,0.009091,0.306452,0.693548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,10,24,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,0,0,0,0,0,0.388889,924.000000,0.016042,0.526316,0.473684
14936,train_14936,8,8,29,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,0,1,0,0,0,0.500000,0.000000,0.000000,0.500000,0.500000
14937,train_14937,22,12,26,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,0,0,0,0,0,0.208333,1105.000000,0.013952,0.916667,0.083333
14938,train_14938,31,12,28,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,0,0,0,0,0,0.000000,2965.000000,0.026568,0.242424,0.757576


# 특성 공학(Feature Engineering)

In [345]:
train_ft['최소구매액']

,최소구매액
0,20000
1,5600
2,4400
3,10000
4,7640
...,...
14935,12000
14936,52000
14937,6000
14938,1000


- ID 변수 제외

In [346]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 494), (12225, 494))

- 추가 피처 만들어 보기

In [347]:
cols = [ col for col in train_ft.columns if col.startswith("중_pivot_횟수비율_") ]

In [348]:
train_ft["중분류별_구매횟수비율_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수비율_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 497), (12225, 497))

## Feature Encoding

In [349]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_중분류,246
주구매_대분류,28
최대구매액_대분류,28
최소구매액_대분류,28


In [350]:
%pip install category_encoders

In [351]:
import category_encoders as ce

In [352]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","주구매_대분류","최소구매액_대분류","최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","주구매_대분류","최소구매액_대분류","최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 585), (12225, 585))

In [353]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 586), (12225, 586))

- 문자열 피처 삭제

In [354]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류', '최대구매액_대분류', '최소구매액_대분류']

In [355]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 581), (12225, 581))

In [356]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [357]:
train_ft.head()

,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28,주구매_중분류_cnt
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,0,0,0,0,0,0,0,0,0,241
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,0,0,0,0,0,0,0,0,0,625
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,0,0,0,0,0,0,0,0,0,26
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,0,0,0,0,0,0,0,0,0,4226
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,0,0,0,0,0,0,0,0,0,53


In [358]:
train_ft['interaction_18시이후구매_주구매지점'] = train_ft['18시이후구매비율'] * train_ft['주구매지점_2']
test_ft['interaction_18시이후구매_주구매지점'] = test_ft['18시이후구매비율'] * test_ft['주구매지점_2']

## Feature Scaling

In [359]:
np.isinf(train_ft).sum().sum(), np.isinf(test_ft).sum().sum()

(1, 1)

In [360]:
train_ft.replace([np.inf, -np.inf], 0, inplace=True)
test_ft.replace([np.inf, -np.inf], 0, inplace=True)

In [361]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [362]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28,주구매_중분류_cnt,interaction_18시이후구매_주구매지점
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497,-0.726970,-0.427107
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497,-0.510078,1.281305
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497,-0.848408,-0.427107
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497,1.523851,-0.427107
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497,-0.833157,-0.427107


# 정답 데이터

In [363]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# cv 점수 확인해보기

In [364]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

# 피처셀렉션

In [365]:
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from lightgbm import LGBMClassifier
# # # 예시 데이터셋 불러오기
# # # X: 피처 데이터셋, y: 타겟 변수
# # # X, y = 데이터셋 로드

# # 데이터셋 분리 (train/test split)
# X_train, X_test, y_train, y_test = train_test_split(train_ft, target, test_size=0.2, random_state=42)

# # 랜덤 포레스트 모델 초기화
# model = RandomForestClassifier(n_estimators=100, random_state=42)

# # 모델 학습
# model.fit(X_train, y_train)

# # 피처 중요도 계산
# feature_importances = model.feature_importances_

# # 피처 중요도를 데이터프레임으로 변환하여 정렬
# feature_importance_df = pd.DataFrame({
#     'Feature': train_ft.columns,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)

# print(feature_importance_df)

In [366]:
# print(feature_importance_df.head(30))

In [367]:
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score

# # 임계값 후보 리스트
# thresholds = np.arange(0.0001, 0.001, 0.0001)  # 0.001 ~ 0.02 사이의 값들을 시도
# best_score = 0
# best_threshold = 0
# best_features = None

# for threshold in thresholds:
#     # 현재 임계값 기준으로 피처 선택
#     selected_features = feature_importance_df[feature_importance_df['Importance'] >= threshold]['Feature'].tolist()
#     X_train_selected = X_train[selected_features]

#     # 모델 학습 및 교차 검증으로 성능 평가
#     model = LGBMClassifier(n_estimators=100, random_state=42)
#     scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')
#     mean_score = np.mean(scores)

#     # 최적의 성능을 찾으면 갱신
#     if mean_score > best_score:
#         best_score = mean_score
#         best_threshold = threshold
#         best_features = selected_features

# print("최적의 임계값:", best_threshold)
# print("최적의 성능:", best_score)
# print("선택된 피처 수:", len(best_features))
# print("선택된 피처 목록:", best_features)

In [368]:
# train_ft=train_ft[best_features]
# test_ft=test_ft[best_features]

In [369]:
# from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# rf = LGBMClassifier(random_state=42) # 특성 선택을 하기 위한 모델
# model = LogisticRegression(random_state=42)


In [370]:
# from sklearn.feature_selection import SelectKBest, SelectPercentile
# model = LGBMClassifier(random_state=42)
# sp = SelectPercentile(percentile=30) # 상위 10% 피처 선택
# x = sp.fit_transform(train_ft, target)
# scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
# scores.mean()

In [371]:
# from sklearn.feature_selection import SelectKBest, SelectPercentile
# from tqdm.notebook import tqdm
# model = LGBMClassifier(random_state=42)

# scores = []
# for percentile in tqdm(range(74,82)):
#     sp = SelectPercentile(percentile=percentile)
#     x = sp.fit_transform(train_ft, target)
#     score = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
#     scores.append([percentile, score])
# scores = np.array(scores)
# idx = np.argmax(scores[:,1])
# best_score = scores[idx]
# import matplotlib.pyplot as plt
# plt.plot(scores[:,0], scores[:,1])
# plt.show()

In [372]:
# sp = SelectPercentile(percentile=best_score[0])
# sp.fit(train_ft,target)
# best_cols = sp.get_feature_names_out()
# print(best_cols)

In [373]:
# from sklearn.feature_selection import SelectFromModel
# train_ft.columns = train_ft.columns.str.replace("  ", "__", regex=False)
# fs = SelectFromModel(rf) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
# x_train = fs.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
# scores = cross_val_score(model, x_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
# scores.mean()

In [374]:
# best_cols = sp.get_feature_names_out()
# best_cols

In [375]:
# train_ft = train_ft[best_cols]
# test_ft = test_ft[best_cols]
# train_ft.shape, test_ft.shape

In [376]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train_ft, target, test_size=0.2, random_state=42)

# model = LGBMClassifier(n_estimators=100, random_state=42)

# model.fit(X_train, y_train)

# feature_importances = model.feature_importances_

# # 피처 중요도를 데이터프레임으로 변환하여 정렬
# feature_importance_df = pd.DataFrame({
#     'Feature': train_ft.columns,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)

# print(feature_importance_df)

In [377]:
# rows_to_drop = feature_importance_df[feature_importance_df["Importance"]==0].index
# feature_importance_df.drop(rows_to_drop, inplace=True)

In [378]:
# cols = feature_importance_df['Feature'].tolist()
# train_ft = train_ft[cols]
# test_ft = test_ft[cols]
# train_ft.shape, test_ft.shape

In [379]:
# feature_importance_df.head(30)

In [380]:
# from sklearn.feature_selection import SelectFromModel
# from lightgbm import LGBMClassifier

# lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
#             reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

# embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
# embeded_lgb_selector.fit(train_ft, target)

In [381]:
# embeded_lgb_support = embeded_lgb_selector.get_support()
# embeded_lgb_feature = train_ft.loc[:,embeded_lgb_support].columns.tolist()
# print(str(len(embeded_lgb_feature)), 'selected features')

#테스트

In [382]:
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(random_state=SEED)
# scores = cross_val_score(model,train_ft,target,cv  = cv ,scoring='f1_macro',n_jobs = -1)
# np.mean(scores)

In [383]:
# from xgboost import XGBClassifier

# model = XGBClassifier(random_state=SEED)
# scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
# np.mean(scores)

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

In [ ]:
# %pip install dask[dataframe]

In [ ]:
# %pip install flaml

In [ ]:
# from flaml import AutoML
# auto_ml = AutoML()
# params = {
#     "metric": "macro_f1",
#     "task" : "classification",
#     "time_budget": 60*3,
#     "seed" : 42,
#     "early_stop": True
# }

# auto_ml.fit(train_ft, target, **params)

In [ ]:
# auto_ml.model.estimator

In [ ]:
# 1-auto_ml_ens.best_loss

In [ ]:
# #앙상블 해보기
# from flaml import AutoML
# auto_ml_ens = AutoML()
# params = {
#     "metric": "macro_f1",
#     "task" : "classification",
#     "time_budget": 60*3,
#     "seed" : SEED,
#     "early_stop": True, # 메타모델이 로지스틱회귀!!
#     "ensemble": True
# }
# auto_ml_ens.fit(train_ft, target, **params)

In [ ]:
# pred = auto_ml_ens.predict(test_ft)
# pred

# 모델 학습

In [ ]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

# 테스트 데이터 예측

In [ ]:
pred = model.predict(test_ft)
pred

In [ ]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit

In [ ]:
submit["target"] = pred
submit

- 예측 결과를 csv 파일로 저장하여 제출

In [ ]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)

In [ ]:
# pred_proba = auto_ml_ens.predict_proba(test_ft)[:,1]
# pred_proba

In [ ]:
submit_proba["target"]=pred_proba
submit_proba

In [ ]:
submit_proba.to_csv(f"{DATA_PATH}submit_proba.csv",index=False)